In [189]:
#Import the necessary libraries for the deployment
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Set the location of the data and its saving process into the code.
page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wikipedia_url = requests.get(page).text
soup = BeautifulSoup(wikipedia_url,'lxml')

#Define the table to work with.
my_table = soup.find('table',{'class':'wikitable sortable'})

In [190]:
#Take the data from the format supported by Soup
A = []
B = []
C = []
for i in my_table.findAll('tr')[1:]:
    links = i.findAll('td')
    A.append(links[0].find(text=True))
    B.append(links[1].find(text=True))
    C.append(links[2].find(text=True).rstrip()) #This clean the \n characters

In [191]:
#Join the data into a DataFrame to further processing
df = pd.DataFrame(A,columns=['PostalCode'])
df['Borough'] = B
df['Neighborhood'] = C

In [192]:
#Delete the Not assigned rows in the df.
df = df.drop(df[df.Borough == 'Not assigned'].index)

In [193]:
#Join the elements by the repeated PostalCode values, and reset the index to further analysis.
df = df.groupby(['PostalCode','Borough'],sort=False).agg( ', '.join)
df = df.reset_index()

In [194]:
#Copy Borough to the Neighborhoods that have Not assigned after all the process.
df.Neighborhood[df[df.Neighborhood == 'Not assigned'].index] = df.Borough[df[df.Neighborhood == 'Not assigned'].index]

In [195]:
#Print the final shape of the df.
df.shape

(103, 3)

In [196]:
#Download the data
!wget -q -O 'coordinates.csv' https://cocl.us/Geospatial_data

In [197]:
#Save the DF
coord_df = pd.read_csv('coordinates.csv')

In [198]:
#Join Both df for further analysis
D = []
E = []
for j in df.PostalCode:
    D.append(coord_df.Latitude[coord_df[coord_df['Postal Code']==j].index].item())
    E.append(coord_df.Longitude[coord_df[coord_df['Postal Code']==j].index].item())
df['Latitude'] = D
df['Longitude'] = E

In [199]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
